<a href="https://colab.research.google.com/github/DikshaJoshi5639/Operational-Efficiency-/blob/main/Optimize_Resource_Allocation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Automated Scheduling

In [ ]:
pip install tensorflow pandas numpy

In [ ]:
pip install prettytable


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import datetime
from prettytable import PrettyTable

# Load the data
data = pd.read_csv('/content/BankCustomerData.csv')

# Preprocess the data
label_encoders = {}
for column in ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le

# Define features and target
X = data.drop(columns=['term_deposit'])
y = data['term_deposit']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a RandomForest model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
print(f"Model Accuracy: {accuracy_score(y_test, y_pred)}")

# Function to predict future demand and allocate resources
def predict_and_allocate(data, model, label_encoders, columns):
    # Preprocess the input data
    for column in label_encoders:
        if column in data:
            le = label_encoders[column]
            data[column] = le.transform(data[column])

    # Ensure the new data has the same columns as the training data
    data = data[columns]

    # Predict future demand
    predictions = model.predict(data)

    # Allocate resources based on predictions
    allocation = []
    for i, prediction in enumerate(predictions):
        if prediction == 1:
            allocation.append('Allocate more resources')
        else:
            allocation.append('Allocate normal resources')

    return allocation

# Function to optimize scheduling and shifts
def optimize_shifts(demand_predictions):
    peak_hours = demand_predictions['hour'].value_counts().idxmax()
    shift_schedule = {}
    for hour in range(24):
        if hour == peak_hours:
            shift_schedule[hour] = 'High demand - allocate more manpower'
        else:
            shift_schedule[hour] = 'Normal demand - allocate normal manpower'
    return shift_schedule

# Function to print allocation in receipt format
def print_allocation_receipt(data):
    table = PrettyTable()
    table.field_names = ["Timestamp", "Hour", "Job", "Resource Allocation"]

    for index, row in data.iterrows():
        table.add_row([row['timestamp'], row['hour'], row['job'], row['allocation']])

    print(table)

# Example of real-time data input
new_data = pd.DataFrame({
    'age': [30, 40],
    'job': ['admin.', 'technician'],
    'marital': ['married', 'single'],
    'education': ['secondary', 'tertiary'],
    'default': ['no', 'no'],
    'balance': [1500, 2000],
    'housing': ['yes', 'yes'],
    'loan': ['no', 'no'],
    'contact': ['cellular', 'cellular'],
    'day': [5, 5],
    'month': ['may', 'may'],
    'duration': [300, 400],
    'campaign': [1, 1],
    'pdays': [999, 999],
    'previous': [0, 0],
    'poutcome': ['unknown', 'unknown']
})

# Add a timestamp to simulate real-time data input
new_data['timestamp'] = [datetime.datetime.now(), datetime.datetime.now() + datetime.timedelta(hours=1)]
new_data['hour'] = new_data['timestamp'].apply(lambda x: x.hour)

# Get the columns used for training
columns = X_train.columns

# Predict demand and allocate resources
resource_allocation = predict_and_allocate(new_data, model, label_encoders, columns)
new_data['allocation'] = resource_allocation

# Optimize shifts based on demand predictions
shift_schedule = optimize_shifts(new_data)

# Print the allocation in receipt format
print_allocation_receipt(new_data)

print(shift_schedule)


Model Accuracy: 0.9180347091932458
+----------------------------+------+-----+---------------------------+
|         Timestamp          | Hour | Job |    Resource Allocation    |
+----------------------------+------+-----+---------------------------+
| 2024-06-27 15:29:41.026983 |  15  |  0  | Allocate normal resources |
| 2024-06-27 16:29:41.026986 |  16  |  9  | Allocate normal resources |
+----------------------------+------+-----+---------------------------+
{0: 'Normal demand - allocate normal manpower', 1: 'Normal demand - allocate normal manpower', 2: 'Normal demand - allocate normal manpower', 3: 'Normal demand - allocate normal manpower', 4: 'Normal demand - allocate normal manpower', 5: 'Normal demand - allocate normal manpower', 6: 'Normal demand - allocate normal manpower', 7: 'Normal demand - allocate normal manpower', 8: 'Normal demand - allocate normal manpower', 9: 'Normal demand - allocate normal manpower', 10: 'Normal demand - allocate normal manpower', 11: 'Normal de

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score
import datetime
from prettytable import PrettyTable

# Load the data
data = pd.read_csv('/content/BankCustomerData.csv')

# Preprocess the data
label_encoders = {}
for column in ['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome']:
    le = LabelEncoder()
    data[column] = le.fit_transform(data[column])
    label_encoders[column] = le

# Standardize numerical features
scaler = StandardScaler()
data[['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']] = scaler.fit_transform(data[['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']])

# Define features and target
X = data.drop(columns=['term_deposit'])
y = data['term_deposit']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Hyperparameter tuning for RandomForest with reduced search space
param_dist = {
    'n_estimators': [50, 100],
    'max_depth': [10, 20],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'bootstrap': [True]
}

# Randomized search with fewer iterations
rf = RandomForestClassifier(random_state=42)
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=param_dist, n_iter=10, cv=3, verbose=2, random_state=42, n_jobs=-1)
rf_random.fit(X_train, y_train)

# Best model
model = rf_random.best_estimator_

# Evaluate the model
y_pred = model.predict(X_test)
print(f"Model Accuracy: {accuracy_score(y_test, y_pred)}")

# Function to predict future demand and allocate resources
def predict_and_allocate(data, model, label_encoders, columns):
    # Preprocess the input data
    for column in label_encoders:
        if column in data:
            le = label_encoders[column]
            data[column] = le.transform(data[column])

    # Standardize numerical features
    data[['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']] = scaler.transform(data[['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']])

    # Ensure the new data has the same columns as the training data
    data = data[columns]

    # Predict future demand
    predictions = model.predict(data)

    # Allocate resources based on predictions
    allocation = []
    for i, prediction in enumerate(predictions):
        if prediction == 1:
            allocation.append('Allocate more resources')
        else:
            allocation.append('Allocate normal resources')

    return allocation

# Function to optimize scheduling and shifts
def optimize_shifts(demand_predictions):
    peak_hours = demand_predictions['hour'].value_counts().idxmax()
    shift_schedule = {}
    for hour in range(24):
        if hour == peak_hours:
            shift_schedule[hour] = 'High demand - allocate more manpower'
        else:
            shift_schedule[hour] = 'Normal demand - allocate normal manpower'
    return shift_schedule

# Function to print allocation in receipt format
def print_allocation_receipt(data):
    table = PrettyTable()
    table.field_names = ["Timestamp", "Hour", "Job", "Resource Allocation"]

    for index, row in data.iterrows():
        table.add_row([row['timestamp'], row['hour'], row['job'], row['allocation']])

    print(table)

# Function to print shift schedule
def print_shift_schedule(schedule):
    table = PrettyTable()
    table.field_names = ["Hour", "Manpower Allocation"]

    for hour, allocation in schedule.items():
        table.add_row([hour, allocation])

    print(table)

# Example of real-time data input
new_data = pd.DataFrame({
    'age': [30, 40],
    'job': ['admin.', 'technician'],
    'marital': ['married', 'single'],
    'education': ['secondary', 'tertiary'],
    'default': ['no', 'no'],
    'balance': [1500, 2000],
    'housing': ['yes', 'yes'],
    'loan': ['no', 'no'],
    'contact': ['cellular', 'cellular'],
    'day': [5, 5],
    'month': ['may', 'may'],
    'duration': [300, 400],
    'campaign': [1, 1],
    'pdays': [999, 999],
    'previous': [0, 0],
    'poutcome': ['unknown', 'unknown']
})

# Add a timestamp to simulate real-time data input
new_data['timestamp'] = [datetime.datetime.now(), datetime.datetime.now() + datetime.timedelta(hours=1)]
new_data['hour'] = new_data['timestamp'].apply(lambda x: x.hour)

# Get the columns used for training
columns = X_train.columns

# Predict demand and allocate resources
resource_allocation = predict_and_allocate(new_data, model, label_encoders, columns)
new_data['allocation'] = resource_allocation

# Optimize shifts based on demand predictions
shift_schedule = optimize_shifts(new_data)

# Print the allocation in receipt format
print_allocation_receipt(new_data)

# Print the shift schedule
print_shift_schedule(shift_schedule)

# Save the final data with resource allocations to a CSV file
new_data.to_csv('/content/ResourceAllocation.csv', index=False)

# Save the shift schedule to a CSV file
shift_schedule_df = pd.DataFrame(list(shift_schedule.items()), columns=['Hour', 'Manpower Allocation'])
shift_schedule_df.to_csv('/content/ShiftSchedule.csv', index=False)


Fitting 3 folds for each of 10 candidates, totalling 30 fits
Model Accuracy: 0.9185037523452158
+----------------------------+------+-----+---------------------------+
|         Timestamp          | Hour | Job |    Resource Allocation    |
+----------------------------+------+-----+---------------------------+
| 2024-06-28 15:41:47.848563 |  15  |  0  | Allocate normal resources |
| 2024-06-28 16:41:47.848568 |  16  |  9  | Allocate normal resources |
+----------------------------+------+-----+---------------------------+
+------+------------------------------------------+
| Hour |           Manpower Allocation            |
+------+------------------------------------------+
|  0   | Normal demand - allocate normal manpower |
|  1   | Normal demand - allocate normal manpower |
|  2   | Normal demand - allocate normal manpower |
|  3   | Normal demand - allocate normal manpower |
|  4   | Normal demand - allocate normal manpower |
|  5   | Normal demand - allocate normal manpower |
|  6